In [1]:
import pyspark
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [58]:
spotify = [
    ('20190525', 1, 20, 'song1', 'artist1'),
    ('20190525', 1, 21, 'song1', 'artist1'),
    ('20190525', 1, 30, 'song1', 'artist1'),
    ('20190525', 2, 20, 'song1', 'artist1'),
    ('20190525', 2, 23, 'song1', 'artist1'),
    ('20190525', 2, 21, 'song1', 'artist1'),
    ('20190525', 3, 20, 'song1', 'artist1'),
    ('20190525', 3, 21, 'song1', 'artist1'),
    ('20190525', 4, 21, 'song1', 'artist1'),
    
    ('20190526', 1, 21, 'song1', 'artist1'),
    ('20190526', 1, 20, 'song1', 'artist1'),
    ('20190526', 1, 30, 'song1', 'artist1'),
    ('20190526', 2, 25, 'song1', 'artist1'),
    ('20190526', 2, 21, 'song1', 'artist1'),
    ('20190526', 2, 30, 'song1', 'artist1'),
    ('20190526', 3, 29, 'song1', 'artist1'),
    ('20190526', 3, 21, 'song1', 'artist1'),
    ('20190526', 4, 21, 'song1', 'artist1'),
    ('20190526', 5, 21, 'song1', 'artist1'),
    ('20190526', 6, 21, 'song1', 'artist1'),
    
    ('20190523', 6, 21, 'song1', 'artist1'),
    ('20190523', 7, 21, 'song1', 'artist1'),
    
]

In [80]:
tags_ = [
    (20, 'rock'),
    (21, 'pop'),
    (23, 'clasica'),
    (22, 'salsa'),
    (25, 'pop'),
    (29, 'rock'),
    (30, 'cumbia'),
]
tags = sc.parallelize(tags_);

In [59]:
data = sc.parallelize(spotify);
ayer = '20190525'
hoy = '20190526'

In [41]:

rss1 = data.filter(lambda x: x[0] >= ayer and x[0] <= hoy)\
            .map(lambda x: ((x[2], x[0]), 1))\
            .reduceByKey(lambda x, y: x+y)
rss1.collect()

[((30, '20190525'), 1),
 ((23, '20190525'), 1),
 ((30, '20190526'), 2),
 ((20, '20190525'), 3),
 ((21, '20190526'), 6),
 ((25, '20190526'), 1),
 ((29, '20190526'), 1),
 ((21, '20190525'), 4),
 ((20, '20190526'), 1)]

In [68]:
hoy = data.filter(lambda x: x[0] == '20190526')\
            .map(lambda x: (x[2], 1))\
            .reduceByKey(lambda x,y: x+y)

ayer = data.filter(lambda x: x[0] == '20190525' )\
            .map(lambda x: (x[2], 1))\
            .reduceByKey(lambda x,y: x+y)

In [73]:
hoy.collect()

[(20, 1), (21, 6), (25, 1), (29, 1), (30, 2)]

In [74]:
ayer.collect()

[(20, 3), (21, 4), (30, 1), (23, 1)]

In [78]:
join = ayer.join(hoy)
join.collect()

[(20, (3, 1)), (21, (4, 6)), (30, (1, 2))]

In [89]:
rising = join.filter(lambda x: x[1][1] > x[1][0])\
            .map(lambda x: (x[0], 'rising'))

In [90]:
                 
new_tags = tags.union(rising)

In [91]:
new_tags.collect()

[(20, 'rock'),
 (21, 'pop'),
 (23, 'clasica'),
 (22, 'salsa'),
 (25, 'pop'),
 (29, 'rock'),
 (30, 'cumbia'),
 (21, 'rising'),
 (30, 'rising')]